<a href="https://colab.research.google.com/github/pberiaintelle/Coviability-indicators/blob/main/PlanetTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=sESGyoKW38K9q-uTP9dspjTOtAbgLWaoOjf02YSTecM&tc=IMdYi1LJ2dodzDKLf0KtcyuoudvDZAMeEBQfdwzxyiE&cc=86RaAsl-_qm4ZMibn_Y3eNbsmna9LrC-u1q_G6KNXV0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BUenG9Almd6VwmJnOSvpvoRZzNf0o4oqwKr6-5swHirPCt-a2c-OKc

Successfully saved authorization token.


In [ ]:
import pandas as pd
import altair as alt
import numpy as np
import folium

In [ ]:
# Make lat. and long. vars an `ee.Geometry.Point`.
aoi = ee.Geometry.Rectangle([-34.740, -6.369, -35.633, -7.709])
#Map.setCenter(-35,-7, 9) # Paraiba coast

In [ ]:
#date parameters
start = ee.Date('2015-12-01');
end_dry = ee.Date('2021-10-29');
start_wet = ee.Date('2021-10-30');
end = ee.Date('2023-05-30');
# Define a function to add year as an image property.
def set_year(img):
  year = ee.Image(img).date().get('year')
  return img.set('Year', year)
# Define a function to get and rename bands of interest from OLI.
def rename_pl(img):
  return (img.select(
      ee.List(['B', 'G', 'R', 'N']),
      ee.List(['blue','green','red','nir'])))

# Define a function to calculate NBR.
def calc_ndvi(img):
  return img.normalizedDifference(ee.List(['nir', 'red'])).rename('NDVI')
#Define function to clip image
def clip(img, aoi):
  clipped_img= mask(img, aoi, crop=True)
  return
  # Define a function to prepare OLI images.
def prep_pl(img):
  orig = img
  img = img.clip(aoi)
  img = rename_pl(img)
  img = calc_ndvi(img)
  img = img.copyProperties(orig, orig.propertyNames())
  return set_year(img)


In [ ]:
#Now for Planet
planet = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')
# Filter collections and prepare them for merging.
planet_col = planet.filterBounds(aoi).map(prep_pl)


In [ ]:
print(planet_col.size().getInfo())

45


In [ ]:
planet_NDVI = planet_col.select("NDVI").median().set("system:time_start", ee.Date("2015"))

In [ ]:
print(planet_NDVI.size().getInfo())

In [ ]:
!pip install geemap
import geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 117.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 124.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 104.3 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11123 sha256=7d565c76c26ab5c493d8dcedd4a37be4ec93f87f5236cf9429766850d370721d
  Stored in directory: /root/.cache/pip/wheels/04/24/fe/140a94a7f1036003ede94579e6b4

In [ ]:
#Visualize image
Map = geemap.Map()
# Load an image.

# Define the visualization parameters.
ndviViz = {'min': 0.0, 'max': 1, 'palette': ['yellow', 'green']}
# Center the map and display the image.
Map.setCenter(-35,-7, 9) # Paraiba coast
Map.addLayer(planet_NDVI, ndviViz, 'NDVI', False)
# Display the map
Map

Map(center=[-7, -35], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

# Reduce to fishnet regions

In [ ]:
pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 37.2 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
from shapely.geometry import mapping
from shapely.geometry import shape

## Load fishnet

In [ ]:
from google.colab import files
a= files.upload()

Saving fishnet.geojson to fishnet.geojson


In [ ]:
geojson_file = 'fishnet.geojson'
gdf = gpd.read_file(geojson_file)
# Print the first few rows of the GeoDataFrame
print(gdf.head())

# Access the geometry column (contains the polygons)
geometries = gdf['geometry']

# Access other attribute columns
attributes = gdf.drop(columns=['geometry'])
# Convert GeoDataFrame polygons to a list of Shapely polygons
shapely_polygons = [shape(geometry) for geometry in gdf['geometry']]

# Convert Shapely polygons to Earth Engine Geometry.Polygon
ee_polygons = [ee.Geometry.Polygon(list(poly.exterior.coords)) for poly in shapely_polygons]

# Convert the list of Earth Engine polygons to an Earth Engine Geometry.MultiPolygon
ee_multi_polygon = ee.Geometry.MultiPolygon(ee_polygons)

# Convert the Earth Engine MultiPolygon to a list of Earth Engine Polygons
ee_polygon_list = ee_multi_polygon.geometries()

# Convert the Earth Engine Polygon list to an Earth Engine FeatureCollection
ee_feature_collection = ee.FeatureCollection(ee_polygon_list)

# Print the resulting Earth Engine Geometry.MultiPolygon
print("ee.Geometry.MultiPolygon:", ee_feature_collection)

Streaming output truncated to the last 5000 lines.
                                      -34.983,
                                      -7.569
                                    ],
                                    [
                                      -35.033,
                                      -7.569
                                    ],
                                    [
                                      -35.033,
                                      -7.519
                                    ]
                                  ]
                                ]
                              },
                              "evenOdd": {
                                "constantValue": true
                              }
                            }
                          }
                        },
                        {
                          "functionInvocationValue": {
                            "functionName": "GeometryConstructors.Polygon",
       

## Reduce data per year

In [ ]:
i = planet_col.first()
band_names = i.bandNames()
print('Band names:', band_names.getInfo())
# Get a list of all metadata properties.
properties = i.propertyNames()
print('Metadata properties:',
      properties.getInfo())  # ee.List of metadata properties
# Get the timestamp.
ee_date = ee.Date(i.get('Year'))
print('Year', ee_date.getInfo())  # ee.Date


Band names: ['NDVI']
Metadata properties: ['Year', 'system:asset_size', 'cadence', 'system:index', 'system:time_start', 'system:footprint', 'system:time_end', 'system:version', 'system:id', 'system:bands', 'system:band_names']
Year {'type': 'Date', 'value': 2015}


In [ ]:
def reduce_year(image_col):
    # Create an empty list to hold yearly mean images
    image_col_year = []

    # Iterate through each year
    for year in range(start_year, end_year + 1):
        # Filter images for the current year
        filtered_images = image_col.filterMetadata('Year', 'equals', year)

        # Calculate the mean if images are available for the year
        if filtered_images.size().getInfo() > 0:
            yearly_mean_image = filtered_images.mean()

            # Set the 'Year' property for the yearly mean image
            yearly_mean_image = yearly_mean_image.set('Year', year)

            image_col_year.append(yearly_mean_image)

    # Create an ImageCollection from the list of yearly mean images
    return ee.ImageCollection(image_col_year)


In [ ]:
start_year=2015
end_year=2023
planet_year = reduce_year(planet_col)

In [ ]:
print(planet_year.size().getInfo())
2023-2015

9


8

In [ ]:
i = planet_year.first()
band_names = i.bandNames()
print('Band names:', band_names.getInfo())
# Get a list of all metadata properties.
properties = i.propertyNames()
print('Metadata properties:',
      properties.getInfo())  # ee.List of metadata properties
# Get the timestamp.
ee_date = ee.Date(i.get('Year'))
print('Year', ee_date.getInfo())  # ee.Date

Band names: ['NDVI']
Metadata properties: ['Year', 'system:index', 'system:bands', 'system:band_names']
Year {'type': 'Date', 'value': 2015}


## Reduce per grid cell

In [ ]:
def reduce_to_grid(polygons, image_col):
    # Initialize an empty list to hold the features for the DataFrame
    features = []

    # Convert the list of Shapely polygons to an Earth Engine Geometry.MultiPolygon
    ee_polygons = [ee.Geometry.Polygon(list(poly.exterior.coords)) for poly in polygons]
    ee_multi_polygon = ee.Geometry.MultiPolygon(ee_polygons)

    # Iterate through each image in the image collection
    for image in image_col.getInfo()['features']:
        image_id = image['properties']['system:index']
        ee_image = ee.Image(image_id)
        year = image['properties']['Year']

        # Reduce the image within the entire MultiPolygon
        cell_mean = ee_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ee_multi_polygon,
            scale=5,  # Replace with desired scale
            maxPixels=1e13
        )

        # Create a dictionary containing cell properties
        cell_properties = {
            'mean_value': cell_mean.get('NDVI'),  # Adjust 'NDVI' to the band name
            'Year': year
        }

        # Append the cell properties dictionary to the list
        features.append(cell_properties)

   # Create a DataFrame from the list of cell properties
    df = pd.DataFrame(features)
    return df

In [ ]:
# Call the function to get the geodataframe with gridded mean values
result_geodataframe = reduce_to_grid(ee_multi_polygon, planet_year)

EEException: ignored